In [3]:
import os, errno, time
from pathlib import Path

# file system types considered risky
RISKY_FS = {"fuse.s3fs", "fuse.rclone", "fuse.sshfs", "panfs", "nfs", "cifs", "smbfs"}

def get_fstype(path: str) -> str | None:
    """
    Best-effort: parse /proc/self/mountinfo to find the fs type for 'path'.
    Returns the fs type string or None if not found/unsupported OS.
    """
    try:
        path = os.path.abspath(path)
        mount_point = "/"
        fstype = None
        best_len = 0
        with open("/proc/self/mountinfo", "r") as f:
            for line in f:
                # mountinfo: ... mount_point ... - fstype source ...
                parts = line.strip().split()
                if "-" not in parts:
                    continue
                dash = parts.index("-")
                mnt_pt = parts[4]
                fs_type = parts[dash + 1]
                # choose longest matching mount point prefix
                if path.startswith(mnt_pt.rstrip("/") + "/") or path == mnt_pt:
                    if len(mnt_pt) > best_len:
                        best_len = len(mnt_pt)
                        mount_point = mnt_pt
                        fstype = fs_type
        return fstype
    except Exception:
        return None

def write_probe(dirpath: str, timeout_s: float = 2.0) -> bool:
    """
    Try a tiny atomic write+fsync+unlink in dirpath.
    Returns True if it succeeds quickly, False if it errors or stalls.
    """
    p = Path(dirpath) / ".probe_safe_cwd.tmp"
    start = time.time()
    try:
        p.write_bytes(b"ok")
        # fsync directory entry to force metadata write
        fd = os.open(dirpath, os.O_DIRECTORY)
        try:
            os.fsync(fd)
        finally:
            os.close(fd)
        p.unlink(missing_ok=True)
        return (time.time() - start) <= timeout_s
    except OSError as e:
        # classic transport / I/O errors -> not safe
        if e.errno in (errno.ENOTCONN, errno.EIO, errno.ESTALE, errno.EROFS):
            return False
        return False
    except Exception:
        return False

def is_safe_cwd(cwd: str | None = None) -> tuple[bool, str]:
    """
    Returns (safe?, reason).
    Safe means: not on a risky fs type AND the write probe succeeds quickly.
    """
    if cwd is None:
        cwd = os.getcwd()
    fstype = get_fstype(cwd)
    if fstype in RISKY_FS:
        ok = write_probe(cwd)
        return (ok, f"Mounted on risky fs '{fstype}', write_probe={'ok' if ok else 'fail'}")
    # If we can’t determine fs type, still try the probe
    ok = write_probe(cwd)
    if not ok:
        return (False, f"write_probe failed in {cwd} (fs type: {fstype or 'unknown'})")
    return (True, f"fs type: {fstype or 'unknown'}, write_probe ok")

def safe_work_dir(preferred: str | None = None) -> str:
    """
    If preferred (or CWD) is unsafe, return a local fallback under /tmp.
    """
    probe_path = preferred or os.getcwd()
    ok, why = is_safe_cwd(probe_path)
    print(f"[safe_work_dir] {probe_path}: {why}")
    if ok:
        return probe_path
    # fallback
    fallback = Path("/tmp") / os.getenv("USER", "user") / "work"
    fallback.mkdir(parents=True, exist_ok=True)
    print(f"[safe_work_dir] Falling back to: {fallback}")
    return str(fallback)


In [4]:
SAFE_CWD = safe_work_dir()          # checks current directory
# or SAFE_CWD = safe_work_dir("/some/target/dir")
SAFE_CWD

OSError: [Errno 107] Transport endpoint is not connected